In [1]:
import sys
sys.path.append('/')
import torch
import cv2
import numpy as np
from torchvision import models
from efficientnet_pytorch import EfficientNet
from grad_cam import GradCam,GuidedBackpropReLUModel,show_cams,show_gbs,preprocess_image
import os
from PIL import Image

## Visualizaion for VGG19

In [12]:
model = models.vgg19(pretrained=True)
grad_cam = GradCam(model=model, blob_name = 'features', target_layer_names=['4','20','36'], use_cuda=False)
img = cv2.imread('./assets/dog.jpg', 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
inputs = preprocess_image(img)
# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested index.
target_index = None
mask_dic = grad_cam(inputs, target_index)
show_cams(img, mask_dic)
gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=False)
show_gbs(inputs, gb_model, target_index, mask_dic)

## Visualizaion for EfficientNet-B0

In [2]:
def get_resized_size():
    path = '/d01/scholles/other/gigasistemica_sandbox_scholles/dataset/CVAT_2/test/Osteoporose_Grave'
    files = os.listdir(path)
    image = Image.open(os.path.join(path, files[0]))
    w, h = image.size
    resized = (int(w/4), int(h/4))
    return resized

model = EfficientNet.from_pretrained('efficientnet-b7')
state_dict = torch.load('/d01/scholles/other/gigasistemica_sandbox_scholles/saved_models/MapTest_CVAT2_EfficientNet_Batch16_Saudavel_Grave.pth')
model.load_state_dict(state_dict)
grad_cam = GradCam(model=model, blob_name = '_blocks', target_layer_names=['1','10','15'], use_cuda=False)
resized = get_resized_size()
print(resized)
img = cv2.imread('/d01/scholles/other/gigasistemica_sandbox_scholles/dataset/CVAT_1/val/Osteoporose/OPHUB2022-4.jpg', 1)
img = np.float32(cv2.resize(img, (715, 337))) / 255
inputs = preprocess_image(img)
# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested index.
target_index = None
mask_dic = grad_cam(inputs, target_index)
show_cams(img, mask_dic)
gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'MemoryEfficientSwish', use_cuda=False)
show_gbs(inputs, gb_model, target_index, mask_dic)

Loaded pretrained weights for efficientnet-b7
(715, 337)


ValueError: operands could not be broadcast together with shapes (224,224,3) (337,715,3) 